In [ ]:
!git clone https://github.com/AchilleBailly/Driver2Vec.git


fatal: destination path 'Driver2Vec' already exists and is not an empty directory.


In [ ]:
import pandas as pd
import pywt
import numpy as np
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm

df = pd.read_csv("Driver2Vec/sample_data/user_0001/user_0001_highway.csv", index_col=0)

def preprocess(df):
    return (
        df.drop(
            ["FOG",
             "FOG_LIGHTS",
             "FRONT_WIPERS",
             "HEAD_LIGHTS",
             "RAIN",
             "REAR_WIPERS",
             "SNOW",
            ], axis=1
        )
    )


test = preprocess(df)
test.shape

(1000, 31)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

def get_dataset(x):
    return TensorDataset(torch.from_numpy(x))

x = get_dataset(test.to_numpy())
print(x)

## Wavelet transform

In [ ]:
def van_haar(data):
    ncol = data.shape[1]
    nrow = data.shape[0]
    for i in range(ncol):
        cur_col = data[:,i].copy()
        (cA, cD) = pywt.dwt(cur_col, 'haar')
        new_col = np.reshape(np.concatenate((cA,cD), 0),(nrow,1))
        data = np.hstack((data,new_col))
    data = data.reshape(nrow)
    return data

van_haar(test.to_numpy()).shape

(1000, 62)

In [ ]:
pywt.dwt(test, 'haar')

(array([[ 1.62363314e-01,  1.67587920e-01,  2.83549819e+02, ...,
          6.36396103e+01, -4.26207870e-02, -4.61006893e-03],
        [ 1.62363314e-01,  1.67587920e-01,  2.83549819e+02, ...,
          6.36396103e+01, -4.26207870e-02, -4.61006893e-03],
        [ 1.62363314e-01,  1.67587920e-01,  2.83549819e+02, ...,
          6.36396103e+01, -4.26207870e-02, -4.61006893e-03],
        ...,
        [ 1.85205062e+00,  6.20983898e-01,  0.00000000e+00, ...,
          6.36396103e+01, -3.13736208e-03,  1.02045238e+00],
        [ 1.85205062e+00,  6.20983898e-01,  0.00000000e+00, ...,
          6.36396103e+01, -3.13736208e-03,  1.02045238e+00],
        [ 1.83451394e+00,  6.30442066e-01,  0.00000000e+00, ...,
          6.36396103e+01, -2.13369471e-03,  1.02940124e+00]]),
 array([[ 1.62363314e-01, -1.90526875e-01,  2.82135606e+02, ...,
          6.36396103e+01,  3.84753381e-02,  0.00000000e+00],
        [ 1.62363314e-01, -1.90526875e-01,  2.82135606e+02, ...,
          6.36396103e+01,  3.84753381e

## PyTorch

In [ ]:
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

class TemporalBlock(nn.Module):
    def __init__(
            self, 
            n_inputs, 
            n_outputs, 
            kernel_size, 
            stride,
            dilation,
            padding,
            dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(
            nn.Conv1d(
                n_inputs, 
                n_outputs, 
                kernel_size,
                stride=stride, 
                padding=padding,
                dilation=dilation))
        self.pad = Chomp1d(padding)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        
        self.conv2 = weight_norm(
            nn.Conv1d(
                n_outputs, 
                n_outputs, 
                kernel_size,
                stride=stride, 
                padding=padding, 
                dilation=dilation))
        self.net = nn.Sequential(
            self.pad, 
            self.conv1, 
            self.relu, 
            self.dropout,
            self.pad, 
            self.conv2, 
            self.relu, 
            self.dropout)
        
        self.downsample = nn.Conv1d(
            n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, 
                                     out_channels, 
                                     kernel_size, 
                                     stride=1, 
                                     dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, 
                                     dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [ ]:
class TCN(nn.Module):
    def __init__(
            self, 
            input_size, 
            output_size, 
            num_channels, 
            kernel_size, 
            dropout):
        super(TCN, self).__init__()
        
        self.tcn = TemporalConvNet(input_size, 
                                   num_channels, 
                                   kernel_size=kernel_size, 
                                   dropout=dropout)
        
        self.linear = nn.Linear(num_channels[-1], output_size)

    def forward(self, inputs):
        """Inputs have to have dimension (N, C_in, L_in)"""
        y1 = self.tcn(inputs)  # input should have dimension (N, C, L)
        o = self.linear(y1[:, :, -1])
        return o

In [ ]:
input_channels = 31
channel_sizes = [32]
kernel_size = 16
dropout = 0.1
model = TemporalConvNet(input_channels, channel_sizes, kernel_size=kernel_size, dropout=dropout)

In [ ]:
model.train()

TemporalConvNet(
  (network): Sequential(
    (0): TemporalBlock(
      (conv1): Conv1d(31, 32, kernel_size=(16,), stride=(1,), padding=(15,))
      (pad): Chomp1d()
      (relu): ReLU()
      (dropout): Dropout(p=0.1, inplace=False)
      (conv2): Conv1d(32, 32, kernel_size=(16,), stride=(1,), padding=(15,))
      (net): Sequential(
        (0): Chomp1d()
        (1): Conv1d(31, 32, kernel_size=(16,), stride=(1,), padding=(15,))
        (2): ReLU()
        (3): Dropout(p=0.1, inplace=False)
        (4): Chomp1d()
        (5): Conv1d(32, 32, kernel_size=(16,), stride=(1,), padding=(15,))
        (6): ReLU()
        (7): Dropout(p=0.1, inplace=False)
      )
      (downsample): Conv1d(31, 32, kernel_size=(1,), stride=(1,))
    )
  )
)

In [ ]:
model.eval()

TemporalConvNet(
  (network): Sequential(
    (0): TemporalBlock(
      (conv1): Conv1d(31, 25, kernel_size=(16,), stride=(1,), padding=(15,))
      (pad): Chomp1d()
      (relu): ReLU()
      (dropout): Dropout(p=0.1, inplace=False)
      (conv2): Conv1d(25, 25, kernel_size=(16,), stride=(1,), padding=(15,))
      (net): Sequential(
        (0): Chomp1d()
        (1): Conv1d(31, 25, kernel_size=(16,), stride=(1,), padding=(15,))
        (2): ReLU()
        (3): Dropout(p=0.1, inplace=False)
        (4): Chomp1d()
        (5): Conv1d(25, 25, kernel_size=(16,), stride=(1,), padding=(15,))
        (6): ReLU()
        (7): Dropout(p=0.1, inplace=False)
      )
      (downsample): Conv1d(31, 25, kernel_size=(1,), stride=(1,))
    )
    (1): TemporalBlock(
      (conv1): Conv1d(25, 32, kernel_size=(16,), stride=(1,), padding=(30,), dilation=(2,))
      (pad): Chomp1d()
      (relu): ReLU()
      (dropout): Dropout(p=0.1, inplace=False)
      (conv2): Conv1d(32, 32, kernel_size=(16,), stride

In [ ]:
x_pred = torch.t(x.tensors[0]).reshape(1,31,1000).float()

In [ ]:
y_pred = model(x_pred)
y_pred

tensor([[[ 521.4061, 1603.4370,  541.6923,  ..., 1570.1222,    0.0000,
             0.0000],
         [ 703.2567, 1678.9783, 2678.9773,  ...,  470.8023,  470.8023,
           470.8041],
         [1454.1556, 1454.1555, 1634.2556,  ..., 1535.6447, 1685.9524,
          1449.8253],
         ...,
         [ 302.7136,  302.7136,  302.7136,  ...,  301.8962,  301.8964,
           301.8851],
         [1765.0485, 1765.0485, 1847.4037,  ..., 1761.5819, 4362.8086,
          4361.4067],
         [  25.5736,  187.9050,  419.1645,  ...,   24.0146,   24.0140,
            24.0213]]], grad_fn=<ReluBackward0>)